In [1]:
print("Hello World!")

Hello World!


# CA
## Subject:
## Lecturer: 
### Student: Vitalis Smirnovs
### Student ID:
## 2022

#### Document Outline:
1. Introduction;
2. Data Description;
3. Exploratory Data Analysis;
4. Statistical Analysis:
    * Correlation;
    * ANOVA;
    * Simple Regression;
    * ML Regression;
    * ML Decision Tree;
5. Conclusion & Discussion;
6. References.

# 1. INTRODOCTION
The purpose of this project is to explore crop yield in Ireland and environmental factors affecting the yield. <br>
Irish agriculture and crop yeild in Ireland: some stats, weather & international comparison

# 2. DATA DESCRIPTION
Finding relevant data is the fist step in data analysis. Finding good/clean data is the dream of any data analist. <br>

Despite the fact that huge amounts of data is generated every second, agriculturel data is not easy to obtain, especially in Ireland. Many Irish farmers farm "old style" and do not have a digital fingerprint. Data generated is often collected as a part of compulsory schemes where farmers provide data to receive government subsidies or as a part of complience proof. In addition farmers who signed up to Teagasc services participate in annual survey. <br>

Data generated is of good quality but impossible to access by anyone but specially authorised persons. <br>

Desite the difficulties spares data is available on CSO.ie [1]. In this project crop yield data for years 1985-2007 was used [2]. This dataset was enhanced with weather data on rainfall [3] and air temperature (???) [4]. <br>

### 2.1 Crop Yield Dataset
Crop yeild dataset [2] can be found in *data.gov.ie* data repository, which in tern cites CSO.ie as a source.
This dataset is well structured and contains the following fields:<br>

| FIELD        	| DESCRIPTION          	|
|--------------	|----------------------	|
| STATISTIC    	| Code for a Statistic 	|
| Statistic    	| Statistic Name       	|
| TLIST(A1)    	| Year                 	|
| Year         	| Year                 	|
| C02039V02469 	| Crop Code            	|
| Type of Crop 	| Crop Type            	|
| UNIT         	| Unit of measurement  	|
| VALUE        	| Value field          	|



In [2]:
# import libraries that are needed for this project
import numpy as np 
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as pltpy
import seaborn as sns
import scipy.stats as stats
import termcolor
from termcolor import colored
#from astroML.plotting import setup_text_plots
#import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
%matplotlib inline
from scipy.optimize import curve_fit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pickle 

In [3]:
# https://forums.fast.ai/t/jupyter-notebook-enhancements-tips-and-tricks/17064/5
# these are the functions to help to make output look nicer and easier to read
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))                                           # for printing in bold

def printmd(string, color=None):                                               
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)  # printing in colour
    display(Markdown(colorstr))

In [4]:
# data source: https://data.gov.ie/dataset/aqa03-crop-yield-1985-2007/resource/2bebaae8-f9a0-4080-9e67-a1f2e130a943
df1 = pd.read_csv('dataCA2_v2.csv') # read in the data into a dataframe
df1                                      # display dataframe

,Year,Code,Crop,CropArea,Yield_ha,Yield
0,1985,1,"Total wheat, oats and barley",386.2,5.1,1986.0
1,1985,11,Total wheat,77.7,6.4,495.0
2,1985,111,Winter wheat,60.3,6.5,395.0
3,1985,112,Spring wheat,17.4,5.8,100.0
4,1985,12,Total oats,24.9,5.1,126.0
...,...,...,...,...,...,...
386,2007,4,Potatoes,11.7,34.0,399.0
387,2007,5,Turnips,NaN,NaN,NaN
388,2007,61,Sugar beet,NaN,NaN,NaN
389,2007,62,Fodder beet,NaN,NaN,NaN


In [5]:
# have a look at the data, to see what it looks like
printmd("**Table 1: Data info**", color='red')      
print("----------------------------------")
print(df1.info())                                    # print data info on screen

printmd("**Table 2: Data head**", color='red')     
print("----------------------------------")
print(df1.head())                                    # view top of the data set

printmd("**Table 3: Data tail**", color='red')
print("----------------------------------")
print(df1.tail())                                    #view bottom of the data set#
printmd("**Table 4: Data Summary Stats**", color='red')                 # print table numbers and headings
printmd("----------------------------------------------", color='red')  # separate output by a line
print(df1.describe())     

<span style='color:red'>**Table 1: Data info**</span>

----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      391 non-null    int64  
 1   Code      391 non-null    int64  
 2   Crop      391 non-null    object 
 3   CropArea  368 non-null    float64
 4   Yield_ha  368 non-null    float64
 5   Yield     368 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 18.5+ KB
None


<span style='color:red'>**Table 2: Data head**</span>

----------------------------------
   Year  Code                          Crop  CropArea  Yield_ha   Yield
0  1985     1  Total wheat, oats and barley     386.2       5.1  1986.0
1  1985    11                   Total wheat      77.7       6.4   495.0
2  1985   111                  Winter wheat      60.3       6.5   395.0
3  1985   112                  Spring wheat      17.4       5.8   100.0
4  1985    12                    Total oats      24.9       5.1   126.0


<span style='color:red'>**Table 3: Data tail**</span>

----------------------------------
     Year  Code                    Crop  CropArea  Yield_ha  Yield
386  2007     4                Potatoes      11.7      34.0  399.0
387  2007     5                 Turnips       NaN       NaN    NaN
388  2007    61              Sugar beet       NaN       NaN    NaN
389  2007    62             Fodder beet       NaN       NaN    NaN
390  2007     7  Kale and field cabbage       NaN       NaN    NaN


<span style='color:red'>**Table 4: Data Summary Stats**</span>

<span style='color:red'>----------------------------------------------</span>

              Year        Code    CropArea    Yield_ha        Yield
count   391.000000  391.000000  368.000000  368.000000   368.000000
mean   1996.000000   53.529412   60.465761   16.276902   536.256522
std       6.641748   53.407562   85.212043   18.614356   578.540756
min    1985.000000    1.000000    0.000000    0.000000     0.000000
25%    1990.000000    5.000000    8.350000    5.900000    89.525000
50%    1996.000000   13.000000   20.750000    7.200000   305.500000
75%    2002.000000  112.000000   68.600000   12.950000   795.525000
max    2007.000000  132.000000  386.200000   73.700000  2500.900000


In [6]:
# climate data
# https://www.met.ie/climate/available-data/long-term-data-sets/
df2 = pd.read_csv('climateData1.csv') # read in the data into a dataframe
df2                                      # display dataframe


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1850,113.5,86.4,38.6,138.3,59.7,54.5,103.4,76.3,70.5,62.7,113.1,104.0
1,1851,195.0,55.2,79.9,54.7,45.0,101.5,90.3,96.6,49.4,113.4,45.4,68.8
2,1852,161.4,92.4,44.6,44.4,70.2,179.4,68.6,110.1,57.3,88.7,223.2,208.2
3,1853,144.7,43.0,83.5,75.3,33.2,77.5,100.5,82.9,60.2,185.1,106.0,48.4
4,1854,138.6,36.0,37.3,18.8,100.0,110.4,102.6,60.5,47.8,77.8,74.6,94.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2006,57.6,54.6,120.3,47.3,126.2,29.8,55.9,70.8,162.6,145.6,142.3,170.5
157,2007,107.9,107.8,80.3,26.9,65.9,141.3,133.0,100.4,52.8,52.2,75.0,123.1
158,2008,174.3,58.0,113.5,45.1,42.3,101.9,123.8,175.9,108.2,152.4,80.4,73.7
159,2009,152.9,36.5,48.7,120.9,90.7,73.2,174.7,148.4,49.4,129.9,244.9,105.4


In [7]:
# clculate average yearly temperature
df2['averageRain'] = (df2['Jan'] +  df2['Feb'] +  df2['Mar'] 
                  +  df2['May'] +  df2['Jun'] +  df2['Jul'] 
                  +  df2['Aug'] +  df2['Sep'] +  df2['Oct'] 
                  +  df2['Nov'] +  df2['Apr'] +  df2['Dec']) / 12

In [8]:
df2

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
0,1850,113.5,86.4,38.6,138.3,59.7,54.5,103.4,76.3,70.5,62.7,113.1,104.0,85.083333
1,1851,195.0,55.2,79.9,54.7,45.0,101.5,90.3,96.6,49.4,113.4,45.4,68.8,82.933333
2,1852,161.4,92.4,44.6,44.4,70.2,179.4,68.6,110.1,57.3,88.7,223.2,208.2,112.375000
3,1853,144.7,43.0,83.5,75.3,33.2,77.5,100.5,82.9,60.2,185.1,106.0,48.4,86.691667
4,1854,138.6,36.0,37.3,18.8,100.0,110.4,102.6,60.5,47.8,77.8,74.6,94.9,74.941667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2006,57.6,54.6,120.3,47.3,126.2,29.8,55.9,70.8,162.6,145.6,142.3,170.5,98.625000
157,2007,107.9,107.8,80.3,26.9,65.9,141.3,133.0,100.4,52.8,52.2,75.0,123.1,88.883333
158,2008,174.3,58.0,113.5,45.1,42.3,101.9,123.8,175.9,108.2,152.4,80.4,73.7,104.125000
159,2009,152.9,36.5,48.7,120.9,90.7,73.2,174.7,148.4,49.4,129.9,244.9,105.4,114.633333


In [9]:
# keep only years after 1985 & before 2007
df2 = df2[(df2.Year >= 1985)]
df2 = df2[(df2.Year <= 2007)]
df2.head(20)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
135,1985,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
136,1986,136.2,6.5,121.1,74.1,124.0,95.4,68.1,174.7,7.5,99.9,138.4,179.3,102.100000
137,1987,56.0,79.7,101.6,73.6,32.2,107.6,47.5,83.7,111.0,144.5,79.1,100.7,84.766667
138,1988,194.6,79.9,114.7,42.2,83.0,49.4,131.1,120.5,80.2,150.3,55.5,72.8,97.850000
139,1989,89.2,102.2,134.3,83.6,20.9,65.4,23.8,105.8,59.9,147.0,60.0,119.1,84.266667
140,1990,135.3,192.5,38.2,52.0,37.3,101.6,58.4,71.5,39.7,178.6,74.8,121.0,91.741667
141,1991,103.4,89.0,109.3,121.3,7.7,91.9,65.2,39.7,79.0,125.3,120.7,63.2,84.641667
142,1992,66.7,71.6,107.9,103.1,55.8,39.4,101.0,142.3,97.8,56.3,131.6,86.8,88.358333
143,1993,133.2,27.7,67.1,104.3,135.0,95.7,87.8,46.1,135.0,62.5,84.8,187.1,97.191667
144,1994,148.3,154.0,119.6,97.9,67.9,55.4,91.9,83.3,87.3,69.7,102.3,159.0,103.050000


In [10]:
df2.describe()


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
count,23.00000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,1996.00000,111.556522,88.991304,87.686957,78.113043,72.026087,77.595652,75.508696,93.834783,83.386957,122.700000,109.578261,117.534783,93.209420
std,6.78233,41.761790,45.292031,29.239797,27.496550,36.383366,32.189954,25.309231,45.646255,39.949563,47.960476,38.987492,41.664980,9.009915
min,1985.00000,35.200000,6.500000,35.700000,26.900000,7.700000,29.800000,23.800000,13.000000,7.500000,52.200000,54.200000,52.000000,77.058333
25%,1990.50000,76.100000,58.100000,65.200000,59.700000,43.850000,53.600000,58.900000,67.900000,58.550000,67.400000,77.050000,88.050000,87.629167
50%,1996.00000,123.400000,79.900000,90.900000,79.900000,67.900000,79.700000,73.900000,83.700000,80.200000,144.500000,102.600000,119.100000,93.866667
75%,2001.50000,140.300000,110.650000,108.600000,100.000000,86.850000,98.700000,87.400000,115.650000,99.850000,160.750000,133.900000,147.100000,98.237500
max,2007.00000,194.600000,192.500000,134.300000,124.000000,139.500000,141.300000,133.000000,191.500000,173.600000,199.100000,187.300000,192.800000,116.108333


In [11]:
# left join in python
 
df3= pd.merge(df1, df2, on='Year', how='left')
df3.head(20)

,Year,Code,Crop,CropArea,Yield_ha,Yield,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
0,1985,1,"Total wheat, oats and barley",386.2,5.1,1986.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
1,1985,11,Total wheat,77.7,6.4,495.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
2,1985,111,Winter wheat,60.3,6.5,395.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
3,1985,112,Spring wheat,17.4,5.8,100.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
4,1985,12,Total oats,24.9,5.1,126.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
5,1985,121,Winter oats,9.1,6.2,56.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
6,1985,122,Spring oats,15.8,4.4,70.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
7,1985,13,Total barley,283.6,4.8,1365.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
8,1985,131,Winter barley,35.5,5.2,185.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
9,1985,132,Spring barley,248.1,4.8,1179.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333


In [12]:
# delete total categories (exclude repeating data)
df3=df3[(df3.Code != 1)]
df3=df3[(df3.Code != 11)]
df3=df3[(df3.Code != 12)]
df3=df3[(df3.Code != 13)]


In [14]:
df3.head(20)

,Year,Code,Crop,CropArea,Yield_ha,Yield,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
2,1985,111,Winter wheat,60.3,6.5,395.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
3,1985,112,Spring wheat,17.4,5.8,100.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
5,1985,121,Winter oats,9.1,6.2,56.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
6,1985,122,Spring oats,15.8,4.4,70.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
8,1985,131,Winter barley,35.5,5.2,185.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
9,1985,132,Spring barley,248.1,4.8,1179.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
10,1985,2,Beans and peas,2.3,0.0,0.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
11,1985,3,Oilseed rape,0.0,0.0,0.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
12,1985,4,Potatoes,32.1,20.8,668.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333
13,1985,5,Turnips,8.7,53.5,466.0,78.6,73.7,107.0,69.0,87.4,79.7,87.0,191.5,97.5,59.9,92.2,124.1,95.633333


In [16]:
corr= np.round(df3.corr(), 2)  # correlation matrix
print(corr)                   # print correlation matrix on screen

          Year  Code  CropArea  Yield_ha  Yield   Jan   Feb   Mar   Apr   May  \
Year      1.00 -0.00     -0.00     -0.09  -0.01 -0.10 -0.01 -0.39 -0.06  0.22   
Code     -0.00  1.00      0.47     -0.40   0.11  0.00  0.00  0.00  0.00  0.00   
CropArea -0.00  0.47      1.00     -0.23   0.56 -0.02 -0.03  0.01 -0.02  0.01   
Yield_ha -0.09 -0.40     -0.23      1.00   0.40  0.02  0.06 -0.01  0.05 -0.07   
Yield    -0.01  0.11      0.56      0.40   1.00 -0.01  0.00 -0.02  0.01 -0.04   
Jan      -0.10  0.00     -0.02      0.02  -0.01  1.00  0.09  0.15  0.07  0.13   
Feb      -0.01  0.00     -0.03      0.06   0.00  0.09  1.00 -0.30 -0.28 -0.22   
Mar      -0.39  0.00      0.01     -0.01  -0.02  0.15 -0.30  1.00  0.09 -0.15   
Apr      -0.06  0.00     -0.02      0.05   0.01  0.07 -0.28  0.09  1.00 -0.12   
May       0.22  0.00      0.01     -0.07  -0.04  0.13 -0.22 -0.15 -0.12  1.00   
Jun       0.01  0.00      0.02     -0.03  -0.01 -0.11  0.01 -0.35  0.00 -0.03   
Jul       0.09 -0.00      0.

In [19]:
# https://towardsdatascience.com/a-beginners-guide-to-linear-regression-in-python-with-scikit-learn-83a8f7ae2b4f
# https://stackabuse.com/linear-regression-in-python-with-scikit-learn/
# https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
X = df3['Yield_ha'] # x variables 
y = df3['averageT']   # y variable

X_train, X_test, y_train, y_test = train_test_split(X, y,              # split data into train and test
                                                    test_size=0.2,     # split 80-20
                                                    random_state=0)
print (X_train.shape, y_train.shape) # view how many lines are in the training set
print (X_test.shape, y_test.shape)   # view how many lines are in the test set

reg = LinearRegression()  # save regression output to a var
reg.fit(X_train, y_train)    # use training set to run regression
y_pred = reg.predict(X_test) # use test set to predict new y 
df4 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}) # view actual y vs predicted values

# save results with pickle
pickle.dump(reg, open('reg.pkl', 'wb'))
load_reg=pickle.load(open('reg.pkl', 'rb'))
pred=[[20]]
result=load_reg.predict(pred)
print(result)

(239,) (239,)
(60,) (60,)


ValueError: Expected 2D array, got 1D array instead:
array=[ 6.8  7.6 54.9  4.4  nan  7.2  6.4  2.7  0.   0.   8.   5.3  nan  6.2
  7.5  nan 64.8  5.  42.   5.9  6.4  7.   5.1  8.   8.1 65.   8.   7.8
  5.4  7.6 33.4  6.   3.1  1.9 57.8  7.4 44.6  7.4  nan  5.   3.5  8.6
 53.1  6.6  4.9  6.6  nan  4.7  7.7  6.9 73.7  5.2  0.   5.5 10.5 22.4
 66.2  4.7  4.3 20.3  5.   6.1  4.8  6.6 45.8  5.1  5.   7.   5.4  5.2
  6.  51.  57.6  7.6  3.3  7.8  7.6  6.7  7.   3.8  5.6  5.5 48.2  4.4
  6.8 30.1 51.1  6.3 49.5  6.4  9.8  0.   7.1 33.7 34.7  7.8 39.   6.2
  nan  6.7 49.8 27.9 38.7  6.5  0.   7.7  5.4 40.8 63.4  8.   6.4  4.5
  8.4 45.2  6.5  5.9  8.7  nan  nan  6.4 34.7 27.6  3.9 33.5  4.5 26.3
  nan  3.1  3.   7.4  8.8  4.  43.7 45.3 52.3  6.5  6.   7.6  nan  8.8
  7.7  7.7 40.1 51.9  3.7  nan 66.8 34.4  6.8  6.4  9.4  8.2 64.8 55.5
 60.   7.   5.1  nan  8.1  0.   5.6 33.6  4.7  7.7  6.8 44.1  3.   9.8
 39.   nan  8.   7.9  nan 56.8  6.9 70.6 56.8  5.5  8.1 41.6  9.   5.8
  4.7  7.4  7.6  5